In [7]:
# Import library
import category_encoders as ce
import matplotlib.pyplot as plt
import miceforest as mf
import missingno as msno
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     cross_val_score, cross_validate,
                                     train_test_split)
from sklearn.tree import DecisionTreeRegressor
from utils import *
from xgboost import XGBRegressor

In [8]:
# set seaborn default style
sns.set(style='darkgrid', palette='muted')

In [11]:
# Import data
df = pd.read_csv('../data/raw/used_car_data.csv')
print('Shape:', df.shape)
df.head()

Shape: (6019, 12)


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


In [13]:
# check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             5983 non-null   object 
 9   Power              5983 non-null   object 
 10  Seats              5977 non-null   float64
 11  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(8)
memory usage: 564.4+ KB


In [14]:
# check null
null_checker(df)

,null (sum),null (%)
Seats,42,0.70
Engine,36,0.60
Power,36,0.60
Mileage,2,0.03
Name,0,0.00
Location,0,0.00
Year,0,0.00
Kilometers_Driven,0,0.00
Fuel_Type,0,0.00
Transmission,0,0.00


In [15]:
# Extrack features
df['Brand'] = df['Name'].apply(lambda x: x.split(' ')[0])
df['Series'] = df['Name'].apply(lambda x: x.split(' ')[1])
df.drop(columns='Name', inplace=True)

In [16]:
# Check features unit
print('Satuan pada feature Mileage:', df['Mileage'].apply(lambda x: x if pd.isna(x) else x.split(' ')[1]).unique())
print('Satuan pada feature Engine:', df['Engine'].apply(lambda x: x if pd.isna(x) else x.split(' ')[1]).unique())
print('Satuan pada feature Power:', df['Power'].apply(lambda x: x if pd.isna(x) else x.split(' ')[1]).unique())

Satuan pada feature Mileage: ['km/kg' 'kmpl' nan]
Satuan pada feature Engine: ['CC' nan]
Satuan pada feature Power: ['bhp' nan]


In [17]:
# Check invalid value
print('Invalid Value pada feature Mileage:', pd.Series([x for x in df['Mileage'] if str(x).split(' ')[0].isalpha()]).unique())
print('Invalid Value pada feature Engine:', pd.Series([x for x in df['Engine'] if str(x).split(' ')[0].isalpha()]).unique())
print('Invalid Value pada feature Power:', pd.Series([x for x in df['Power'] if str(x).split(' ')[0].isalpha()]).unique())

Invalid Value pada feature Mileage: [nan]
Invalid Value pada feature Engine: [nan]
Invalid Value pada feature Power: ['null bhp' nan]


In [18]:
# Remove features unit and convert to numeric
df['Mileage (kmpl)'] = df['Mileage'].apply(lambda x: x if pd.isna(x) else x.split(' ')[0])
df['Engine (CC)'] = df['Engine'].apply(lambda x: x if pd.isna(x) else x.split(' ')[0])
df['Power (bhp)'] = df['Power'].apply(lambda x: x if pd.isna(x) else x.split(' ')[0])

df['Mileage (kmpl)'] = pd.to_numeric(df['Mileage (kmpl)'], errors='coerce')
df['Engine (CC)'] = pd.to_numeric(df['Engine (CC)'], errors='coerce')
df['Power (bhp)'] = pd.to_numeric(df['Power (bhp)'], errors='coerce')

df.drop(columns=['Mileage', 'Engine', 'Power'], inplace=True)

In [19]:
# Check result
df.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Price,Brand,Series,Mileage (kmpl),Engine (CC),Power (bhp)
0,Mumbai,2010,72000,CNG,Manual,First,5.0,1.75,Maruti,Wagon,26.60,998.0,58.16
1,Pune,2015,41000,Diesel,Manual,First,5.0,12.50,Hyundai,Creta,19.67,1582.0,126.20
2,Chennai,2011,46000,Petrol,Manual,First,5.0,4.50,Honda,Jazz,18.20,1199.0,88.70
3,Chennai,2012,87000,Diesel,Manual,First,7.0,6.00,Maruti,Ertiga,20.77,1248.0,88.76
4,Coimbatore,2013,40670,Diesel,Automatic,Second,5.0,17.74,Audi,A4,15.20,1968.0,140.80


In [20]:
# Summary statistic
df.describe()

,Year,Kilometers_Driven,Seats,Price,Mileage (kmpl),Engine (CC),Power (bhp)
count,6019.000000,6.019000e+03,5977.000000,6019.000000,6017.000000,5983.000000,5876.000000
mean,2013.358199,5.873838e+04,5.278735,9.479468,18.134961,1621.276450,113.253050
std,3.269742,9.126884e+04,0.808840,11.187917,4.582289,601.355233,53.874957
min,1998.000000,1.710000e+02,0.000000,0.440000,0.000000,72.000000,34.200000
25%,2011.000000,3.400000e+04,5.000000,3.500000,15.170000,1198.000000,75.000000
50%,2014.000000,5.300000e+04,5.000000,5.640000,18.150000,1493.000000,97.700000
75%,2016.000000,7.300000e+04,5.000000,9.950000,21.100000,1984.000000,138.100000
max,2019.000000,6.500000e+06,10.000000,160.000000,33.540000,5998.000000,560.000000


In [21]:
# Check milage 0
df[df['Mileage (kmpl)']==0]

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Price,Brand,Series,Mileage (kmpl),Engine (CC),Power (bhp)
14,Pune,2012,85000,Diesel,Automatic,Second,5.0,17.50,Land,Rover,0.0,2179.0,115.0
67,Coimbatore,2019,15369,Diesel,Automatic,First,5.0,35.67,Mercedes-Benz,C-Class,0.0,1950.0,194.0
79,Hyderabad,2005,87591,Petrol,Manual,First,5.0,1.30,Hyundai,Santro,0.0,1086.0,NaN
194,Ahmedabad,2007,60006,Petrol,Manual,First,NaN,2.95,Honda,City,0.0,NaN,NaN
229,Bangalore,2015,70436,Diesel,Manual,First,NaN,3.60,Ford,Figo,0.0,1498.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647,Mumbai,2001,227000,Diesel,Manual,Fourth & Above,8.0,2.20,Toyota,Qualis,0.0,2446.0,NaN
5875,Ahmedabad,2019,4000,Diesel,Automatic,First,5.0,35.00,Mercedes-Benz,C-Class,0.0,1950.0,194.0
5943,Chennai,2002,75000,Diesel,Manual,First,6.0,1.70,Mahindra,Jeep,0.0,2112.0,NaN
5972,Mumbai,2008,65000,Petrol,Manual,Second,5.0,1.39,Hyundai,Santro,0.0,1086.0,62.0


In [22]:
# Seats 0 value
df[df['Seats']==0]

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Price,Brand,Series,Mileage (kmpl),Engine (CC),Power (bhp)
3999,Hyderabad,2012,125000,Petrol,Automatic,First,0.0,18.0,Audi,A4,10.5,3197.0,NaN


In [23]:
# Replace 0 value to nan
df['Mileage (kmpl)'] = df['Mileage (kmpl)'].replace(0, np.nan)
df['Seats'] = df['Seats'].replace(0, np.nan)

In [24]:
# Check unique value
cat_cols = [col for col in df.columns if df[col].dtypes == 'object']
df[cat_cols].nunique()

Location         11
Fuel_Type         5
Transmission      2
Owner_Type        4
Brand            31
Series          212
dtype: int64

In [25]:
for col in cat_cols:
  print(col, df[col].unique(), '\n')

Location ['Mumbai' 'Pune' 'Chennai' 'Coimbatore' 'Hyderabad' 'Jaipur' 'Kochi'
 'Kolkata' 'Delhi' 'Bangalore' 'Ahmedabad'] 

Fuel_Type ['CNG' 'Diesel' 'Petrol' 'LPG' 'Electric'] 

Transmission ['Manual' 'Automatic'] 

Owner_Type ['First' 'Second' 'Fourth & Above' 'Third'] 

Brand ['Maruti' 'Hyundai' 'Honda' 'Audi' 'Nissan' 'Toyota' 'Volkswagen' 'Tata'
 'Land' 'Mitsubishi' 'Renault' 'Mercedes-Benz' 'BMW' 'Mahindra' 'Ford'
 'Porsche' 'Datsun' 'Jaguar' 'Volvo' 'Chevrolet' 'Skoda' 'Mini' 'Fiat'
 'Jeep' 'Smart' 'Ambassador' 'Isuzu' 'ISUZU' 'Force' 'Bentley'
 'Lamborghini'] 

Series ['Wagon' 'Creta' 'Jazz' 'Ertiga' 'A4' 'EON' 'Micra' 'Innova' 'Vento'
 'Indica' 'Ciaz' 'City' 'Swift' 'Rover' 'Pajero' 'Amaze' 'Duster' 'New'
 '3' 'S' 'A6' 'i20' 'Alto' 'WRV' 'Corolla' 'Ssangyong' 'Vitara' 'KUV'
 'M-Class' 'Polo' 'Nano' 'Elantra' 'Xcent' 'Thar' 'Grand' 'KWID' 'i10'
 'X-Trail' 'Zen' 'Figo' 'C-Class' 'Cayenne' 'XUV500' 'Terrano' 'Brio'
 'Fiesta' 'Santro' 'Zest' 'Ritz' '5' 'Fortuner' 'Ecosport' 'Verna

In [26]:
# Replace duplicated value
df['Brand'] = df['Brand'].replace('ISUZU', 'Isuzu')

In [27]:
df.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Price,Brand,Series,Mileage (kmpl),Engine (CC),Power (bhp)
0,Mumbai,2010,72000,CNG,Manual,First,5.0,1.75,Maruti,Wagon,26.60,998.0,58.16
1,Pune,2015,41000,Diesel,Manual,First,5.0,12.50,Hyundai,Creta,19.67,1582.0,126.20
2,Chennai,2011,46000,Petrol,Manual,First,5.0,4.50,Honda,Jazz,18.20,1199.0,88.70
3,Chennai,2012,87000,Diesel,Manual,First,7.0,6.00,Maruti,Ertiga,20.77,1248.0,88.76
4,Coimbatore,2013,40670,Diesel,Automatic,Second,5.0,17.74,Audi,A4,15.20,1968.0,140.80


In [28]:
df.describe()

,Year,Kilometers_Driven,Seats,Price,Mileage (kmpl),Engine (CC),Power (bhp)
count,6019.000000,6.019000e+03,5976.000000,6019.000000,5949.000000,5983.000000,5876.000000
mean,2013.358199,5.873838e+04,5.279618,9.479468,18.342252,1621.276450,113.253050
std,3.269742,9.126884e+04,0.806019,11.187917,4.175475,601.355233,53.874957
min,1998.000000,1.710000e+02,2.000000,0.440000,6.400000,72.000000,34.200000
25%,2011.000000,3.400000e+04,5.000000,3.500000,15.300000,1198.000000,75.000000
50%,2014.000000,5.300000e+04,5.000000,5.640000,18.200000,1493.000000,97.700000
75%,2016.000000,7.300000e+04,5.000000,9.950000,21.100000,1984.000000,138.100000
max,2019.000000,6.500000e+06,10.000000,160.000000,33.540000,5998.000000,560.000000


In [29]:
df.describe(include=['object']) 

,Location,Fuel_Type,Transmission,Owner_Type,Brand,Series
count,6019,6019,6019,6019,6019,6019
unique,11,5,2,4,30,212
top,Mumbai,Diesel,Manual,First,Maruti,Swift
freq,790,3205,4299,4929,1211,353


In [30]:
null_checker(df)

,null (sum),null (%)
Power (bhp),143,2.38
Mileage (kmpl),70,1.16
Seats,43,0.71
Engine (CC),36,0.60
Location,0,0.00
Year,0,0.00
Kilometers_Driven,0,0.00
Fuel_Type,0,0.00
Transmission,0,0.00
Owner_Type,0,0.00
